In [10]:
import numpy as np
import os
import matplotlib.pyplot as plt
import os
import time
import glob
import sys
# DATA = os.getenv('DEXNET_DATA')
# print(DATA)


In [32]:
arrays = {}

print("Completion progress bar")
toolbar_width = 100
# setup toolbar
sys.stdout.write("[%s]\n[" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b[" * (toolbar_width+1)) # return to start of line, after '['
perct_value = 0
total_data = 87477
i = 0
files_dir = '/home/ashay/Downloads/dexnet_2/dexnet_2_tensor/tensors'
for filename in os.listdir(files_dir):          
    if filename.endswith('.npz'): 
        with np.load( files_dir + '/' + filename) as arc:
            arrays[filename.replace('.npz', '')] = arc

    perct_comp = i/total_data*toolbar_width
    if perct_value < perct_comp:
        sys.stdout.write("#")
        sys.stdout.flush()
        perct_value += 1
    i += 1
    
sys.stdout.write("]\n")
    
    

# features = {}
# for array in arrays:
#     f = arrays[array]
#     feature = f['arr_0.npy']
#     features[array] = feature
# print(features.keys())



Completion progress bar
[                                                                                                    ]
[####################################################################################################]


In [19]:
print(len(arrays))

87477


In [215]:
# #Inputs to feed into CNN

#-----------x-------------------
search_key1 = 'depth_ims_tf_table'
aligned_imgs_x1 = np.array([val for key, val in features.items() if search_key1 in key])
print(np.shape(aligned_imgs_x1))

search_key2 = 'hand_poses'
gripper_depths_x2 = np.array([val for key, val in features.items() if search_key2 in key])
gripper_depths_x2 = gripper_depths_x2[:, :, 2]
print(np.shape(gripper_depths_x2))


#------------y------------------
search_key3 = 'robust_ferrari_canny'
grasp_metrics_y = np.array([val for key, val in features.items() if search_key3 in key])
print(np.shape(grasp_metrics_y))



(3, 1000, 32, 32, 1)
(3, 1000)
(3, 1000)


In [216]:
#--------merging-----------

aligned_imgs = aligned_imgs_x1[0]
gripper_depths = gripper_depths_x2[0]
grasp_metrics = grasp_metrics_y[0]
for  i in range(1,NUM_EXAMPLES):   #replace the limit with the number of examples
    #-----------------x1------------------
    aligned_imgs = np.concatenate([aligned_imgs,aligned_imgs_x1[i]])
    #-----------------x2-------------------
    gripper_depths = np.concatenate([gripper_depths,gripper_depths_x2[i]])
    #------------------y---------------------
    grasp_metrics = np.concatenate([grasp_metrics,grasp_metrics_y[i]])

print(np.shape(aligned_imgs))
print(np.shape(gripper_depths))
print(np.shape(grasp_metrics))


(3000, 32, 32, 1)
(3000,)
(3000,)


In [217]:
#******************One hot encoding***********************

# print(grasp_metrics[2])
def one_hot_encoding(grasp_metrics):
    for i in range(1000):
        if(grasp_metrics[i] > 0.002):  #threshold value is 0.002 readme
            grasp_metrics[i] = 1
        else:
            grasp_metrics[i] = 0
    return grasp_metrics

grasp_metrics = one_hot_encoding(grasp_metrics)

# print(grasp_metrics)


In [218]:
import tensorflow as tf
import math
import random

# example of defining the discriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Lambda
from tensorflow.nn import local_response_normalization
from keras.utils.vis_utils import plot_model

from keras.layers.merge import concatenate
from keras.models import Model
# from tensorflow.keras import layers
from keras.layers import Dense, Input


#TODO: Initialize weights in layers according to the paper https://arxiv.org/pdf/1703.09312.pdf
def getGraspQualityVariable():
    input = Input(shape=(32, 32, 1), name="img")
    x1 = Conv2D(filters=64, kernel_size=7, activation='relu')(input)
    x2 = Conv2D(filters=64, kernel_size=5, activation='relu')(x1)
    x3 = Lambda(local_response_normalization)(x2)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=2)(x3)
    x5 = Conv2D(filters=64, kernel_size=3, activation='relu')(x4)
    # x6 = Dropout(0.3)(x5)
    x6 = Conv2D(filters=64, kernel_size=3, activation='relu')(x5)
    x7 = Lambda(local_response_normalization)(x6)
    x8 = Flatten()(x7)
    x9 = Dense(1024, activation='relu')(x8)
    
    # plot_model(model, to_file='GraspQualityModel_plot.png', show_shapes=True, show_layer_names=True)
    return x9, input
    
getGraspQualityVariable()


def getPointcloudModel():
    input = Input(shape=(1), name="z")
    x = Dense(16, input_dim=1, activation='relu')(input)
    return x, input


from keras.layers.merge import concatenate
def getDexnet2Model():
    grasp_model, input_1 = getGraspQualityVariable()
    pc_model, input_2 = getPointcloudModel()

    out = Dense(1024, activation='relu')

    # x = layers.concatenate([grasp_model, pc_model])
    merge = concatenate([grasp_model, pc_model])
    out_1 = Dense(1024, activation='relu')(merge)
    out_2 = Dense(2, , activation='softmax')(out_1)
    model = Model(inputs=[input_1, input_2], outputs=out_2)
    plot_model(model, to_file='Dexnet2_plot.png', show_shapes=True, show_layer_names=True)
    return model


In [219]:
first_model = getDexnet2Model()


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [220]:
first_model.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 26, 26, 64)   3200        img[0][0]                        
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 22, 22, 64)   102464      conv2d_44[0][0]                  
__________________________________________________________________________________________________
lambda_22 (Lambda)              (None, 22, 22, 64)   0           conv2d_45[0][0]                  
____________________________________________________________________________________________

In [221]:
first_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [222]:
from keras.callbacks import ModelCheckpoint
start_time = time.time()
checkpointer = ModelCheckpoint(filepath='/home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5', verbose = 1, save_best_only=True)
x_train = [aligned_imgs, gripper_depths]                                   #Check model.summary() in previous section and check model architecture in paper
y_train = grasp_metrics                                                    #valued between [0, 1] grasp robustness for the given grasp
first_model.fit(x_train,
          y_train,
          batch_size=16,                                                   #worth ecperimenting
          epochs=1,                                      
          validation_split=0.25,                                            #Decide on a number
          callbacks=[checkpointer])        
#CHange the filepath of the checkpointer varianble to store different version of weights
elapsed_time = (time.time() - start_time)
print(f"--- {elapsed_time} seconds ---")
print(f"--- {elapsed_time/60} minutes ---")
print(f"--- {elapsed_time/3600} hours ---")



141/141 [==============================] - 8s 52ms/step - loss: 0.9583 - accuracy: 0.7677 - val_loss: 0.6931 - val_accuracy: 0.6360

Epoch 00001: val_loss improved from inf to 0.69315, saving model to /home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5
--- 7.8306121826171875 seconds ---
--- 0.1305102030436198 minutes ---
--- 0.0021751700507269965 hours ---


In [223]:
first_model.predict([aligned_imgs[290:300],gripper_depths[290:300]])

array([[-16.139938, -35.90434 ],
       [-16.138353, -35.8999  ],
       [-16.151684, -35.934444],
       [-16.137676, -35.89809 ],
       [-16.135368, -35.89767 ],
       [-16.14027 , -35.906994],
       [-16.136408, -35.89941 ],
       [-16.156223, -35.944447],
       [-16.142263, -35.911503],
       [-16.136065, -35.89853 ]], dtype=float32)

In [224]:
grasp_metrics[290:300]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])